In [1]:
# 从obs下载数据集
from modelarts.session import Session
session = Session()
session.obs.download_dir(src_obs_dir="obs://mindcon-lldois/food-class/Data/",
                         dst_local_dir="/home/ma-user/work/")

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
100%|██████████| 5000/5000 [00:30<00:00, 164.27it/s]


In [1]:
pip install download

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 127 kB 57.7 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.10
    Uninstalling urllib3-1.26.10:
      Successfully uninstalled urllib3-1.26.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moxing-framework 2.0.0.rc2.4b57a67b requires urllib3>=1.26.2, but you have urllib3 1.25.11 which is incompatible.
modelarts 1.4.16 requires psutil~=5.9.1, but you have psutil 5.7.0 which is incompatible.
modelarts 1.4.16 requires requests~=2.28.1, but you have requests 2.23.0 which is incompatible.
modelarts 1.4.16 requires urllib3~=1.26.11, but you have urllib3 1.25.11 which is incompatible.
ma-cau 1.1.6 requires click==8.0.3, but you have click 8.1.3 which is incompatible.
ma-cau 1.1.6 requires matplotlib==3.5.1, 

In [2]:
"""
LLdois
mindspore 1.7.0
训练模型文件
"""
import mindspore as ms
import mindspore.dataset as ds
import mindspore.dataset.vision.c_transforms as c_vision
from typing import Type, Union, List, Optional
from mindspore import nn
from mindspore.common.initializer import Normal
from mindspore import load_checkpoint, load_param_into_net
from download import download
import os
from mindspore.train.callback import LossMonitor
from mindspore import context
from mindspore.train.callback import Callback


# 加载训练集和测试集
def creat_dataset(dataset_path, workers, image_size, batch_size):
    # 加载数据
    data_set = ds.ImageFolderDataset(dataset_path,
                                     num_parallel_workers=workers,
                                     shuffle=False, )
    # 拆分训练集和验证集
    train_set, val_set = data_set.split([0.8, 0.2])
    # 数据增强操作
    mean = [0.485 * 255, 0.456 * 255, 0.406 * 255]
    std = [0.229 * 255, 0.224 * 255, 0.225 * 255]
    scale = 32
    trans_train = [
        c_vision.RandomCropDecodeResize(size=image_size, scale=(0.08, 1.0),
                                        ratio=(0.75, 1.333)),
        c_vision.RandomHorizontalFlip(prob=0.5),
        c_vision.Normalize(mean=mean, std=std),
        c_vision.HWC2CHW()
    ]
    trans_val = [
        c_vision.Decode(),
        c_vision.Resize(image_size + scale),
        c_vision.CenterCrop(image_size),
        c_vision.Normalize(mean=mean, std=std),
        c_vision.HWC2CHW()
    ]
    # 数据映射操作
    train_set = train_set.map(
        operations=trans_train,
        input_columns='image',
        num_parallel_workers=workers)
    val_set = val_set.map(
        operations=trans_val,
        input_columns='image',
        num_parallel_workers=workers)
    # 批量操作
    train_set = train_set.batch(batch_size)
    val_set = val_set.batch(batch_size)
    # 打乱数据集
    train_set = train_set.shuffle(train_set.get_dataset_size())
    val_set = val_set.shuffle(val_set.get_dataset_size())
    return train_set, val_set


# 构建resnet50网络
class ResidualBlockBase(nn.Cell):
    expansion: int = 1  # 最后一个卷积核数量与第一个卷积核数量相等

    def __init__(self, in_channel: int, out_channel: int,
                 stride: int = 1, norm: Optional[nn.Cell] = None,
                 down_sample: Optional[nn.Cell] = None) -> None:
        super(ResidualBlockBase, self).__init__()
        if not norm:
            self.norm = nn.BatchNorm2d(out_channel)
        else:
            self.norm = norm

        self.conv1 = nn.Conv2d(in_channel, out_channel,
                               kernel_size=3, stride=stride,
                               weight_init=Normal(mean=0, sigma=0.02))
        self.conv2 = nn.Conv2d(in_channel, out_channel,
                               kernel_size=3, weight_init=Normal(mean=0, sigma=0.02))
        self.relu = nn.ReLU()
        self.down_sample = down_sample

    def construct(self, x):
        """ResidualBlockBase construct."""
        identity = x  # shortcuts分支

        out = self.conv1(x)  # 主分支第一层：3*3卷积层
        out = self.norm(out)
        out = self.relu(out)
        out = self.conv2(out)  # 主分支第二层：3*3卷积层
        out = self.norm(out)

        if self.down_sample is not None:
            identity = self.down_sample(x)
        out += identity  # 输出为主分支与shortcuts之和
        out = self.relu(out)

        return out


class ResidualBlock(nn.Cell):
    expansion = 4  # 最后一个卷积核的数量是第一个卷积核数量的4倍

    def __init__(self, in_channel: int, out_channel: int,
                 stride: int = 1, down_sample: Optional[nn.Cell] = None) -> None:
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channel, out_channel,
                               kernel_size=1, weight_init=Normal(mean=0, sigma=0.02))
        self.norm1 = nn.BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(out_channel, out_channel,
                               kernel_size=3, stride=stride,
                               weight_init=Normal(mean=0, sigma=0.02))
        self.norm2 = nn.BatchNorm2d(out_channel)
        self.conv3 = nn.Conv2d(out_channel, out_channel * self.expansion,
                               kernel_size=1, weight_init=Normal(mean=0, sigma=0.02))
        self.norm3 = nn.BatchNorm2d(out_channel * self.expansion)

        self.relu = nn.ReLU()
        self.down_sample = down_sample

    def construct(self, x):
        identity = x  # shortscuts分支

        out = self.conv1(x)  # 主分支第一层：1*1卷积层
        out = self.norm1(out)
        out = self.relu(out)
        out = self.conv2(out)  # 主分支第二层：3*3卷积层
        out = self.norm2(out)
        out = self.relu(out)
        out = self.conv3(out)  # 主分支第三层：1*1卷积层
        out = self.norm3(out)

        if self.down_sample is not None:
            identity = self.down_sample(x)

        out += identity  # 输出为主分支与shortcuts之和
        out = self.relu(out)

        return out


def make_layer(last_out_channel, block: Type[Union[ResidualBlockBase, ResidualBlock]],
               channel: int, block_nums: int, stride: int = 1):
    down_sample = None  # shortcuts分支

    if stride != 1 or last_out_channel != channel * block.expansion:
        down_sample = nn.SequentialCell([
            nn.Conv2d(last_out_channel, channel * block.expansion,
                      kernel_size=1, stride=stride, weight_init=Normal(mean=0, sigma=0.02)),
            nn.BatchNorm2d(channel * block.expansion, gamma_init=Normal(mean=1, sigma=0.02))
        ])

    layers = []
    layers.append(block(last_out_channel, channel, stride=stride, down_sample=down_sample))

    in_channel = channel * block.expansion
    # 堆叠残差网络
    for _ in range(1, block_nums):
        layers.append(block(in_channel, channel))

    return nn.SequentialCell(layers)


class ResNet(nn.Cell):
    def __init__(self, block: Type[Union[ResidualBlockBase, ResidualBlock]],
                 layer_nums: List[int], num_classes: int, input_channel: int) -> None:
        super(ResNet, self).__init__()

        self.relu = nn.ReLU()
        # 第一个卷积层，输入channel为3（彩色图像），输出channel为64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, weight_init=Normal(mean=0, sigma=0.02))
        self.norm = nn.BatchNorm2d(64)
        # 最大池化层，缩小图片的尺寸
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, pad_mode='same')
        # 各个残差网络结构块定义，
        self.layer1 = make_layer(64, block, 64, layer_nums[0])
        self.layer2 = make_layer(64 * block.expansion, block, 128, layer_nums[1], stride=2)
        self.layer3 = make_layer(128 * block.expansion, block, 256, layer_nums[2], stride=2)
        self.layer4 = make_layer(256 * block.expansion, block, 512, layer_nums[3], stride=2)
        # 平均池化层
        self.avg_pool = nn.AvgPool2d()
        # flattern层
        self.flatten = nn.Flatten()
        # 全连接层
        self.fc = nn.Dense(in_channels=input_channel, out_channels=num_classes)

    def construct(self, x):
        x = self.conv1(x)
        x = self.norm(x)
        x = self.relu(x)
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avg_pool(x)
        x = self.flatten(x)
        x = self.fc(x)

        return x


def _resnet(model_url: str, block: Type[Union[ResidualBlockBase, ResidualBlock]],
            layers: List[int], num_classes: int, pretrained: bool, pretrianed_ckpt: str,
            input_channel: int):
    model = ResNet(block, layers, num_classes, input_channel)

    if pretrained:
        # 加载预训练模型
        download(url=model_url, path=pretrianed_ckpt)
        param_dict = load_checkpoint(pretrianed_ckpt)
        load_param_into_net(model, param_dict)

    return model


def resnet50(num_classes: int = 1000, pretrained: bool = False):
    """ResNet50模型"""
    resnet50_url = "https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/models/application" \
                   "/resnet50_224_new.ckpt "
    resnet50_ckpt = "./LoadPretrainedModel/resnet50_224_new.ckpt"
    return _resnet(resnet50_url, ResidualBlock, [3, 4, 6, 3], num_classes,
                   pretrained, resnet50_ckpt, 2048)


class EarlyStop(Callback):
    def __init__(self, epoch):
        super(EarlyStop, self).__init__()
        self.epoch = epoch
        self.loss = 100

    def epoch_end(self, run_context):
        cb_params = run_context.original_args()
        loss = cb_params.net_outputs
        self.loss = loss.asnumpy()
        print("epoch :{},loss:{}".format(self.epoch, loss))
        # if loss.asnumpy() < self._control_loss:
        #     # Stop training
        #     run_context._stop_requested = True


def main():
    # 定义参数
    batch_size = 16  # 批量大小
    image_size = 224  # 训练图像空间大小
    num_epochs = 500  # 最大训练周期数
    lr = 0.001  # 学习率
    momentum = 0.9  # momentum
    workers = 8  # 并行线程个数
    # 使用ascend加速
    context.set_context(mode=context.GRAPH_MODE, device_target="Ascend")
    # 数据集目录路径
    dataset_path = "./Data/train"
    # 加载数据集
    dataset_train, dataset_val = creat_dataset(dataset_path=dataset_path, workers=workers,
                                               image_size=image_size, batch_size=batch_size)
    # 实例化resnet50网络
    network = resnet50(pretrained=True)
    # 模型微调
    # 全连接层输入层的大小
    in_channels = network.fc.in_channels
    # 输出通道数大小为分类数10
    head = nn.Dense(in_channels, 10)
    # 重置全连接层
    network.fc = head

    # 平均池化层kernel size为7
    avg_pool = nn.AvgPool2d(kernel_size=7)
    # 重置平均池化层
    network.avg_pool = avg_pool
    # 定义优化器和损失函数
    opt = nn.Momentum(params=network.trainable_params(), learning_rate=lr, momentum=momentum)
    loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')

    # 调用高阶接口
    model = ms.Model(network, loss_fn, opt, metrics={"Accuracy": nn.Accuracy()})
    # 最佳模型保存路径
    best_ckpt_dir = "./model"
    best_ckpt_path = "./model/resnet50-best.ckpt"
    # # 开始训练
    # loss_monitor = LossMonitor()
    print("开始训练...")
    best_acc = 0
    temp = 0
    for epoch in range(1, num_epochs):
        stop_cb = EarlyStop(epoch=epoch)
        model.train(1, dataset_train, callbacks=[stop_cb])
        if stop_cb.loss <= 0.005:
            print("已结束")
            break
        temp = temp + 1
        if temp % 5 == 0:
            acc = model.eval(dataset_val)['Accuracy']
            print("acc:{}".format(acc))
            if acc >= best_acc:
                best_acc = acc
                if not os.path.exists(best_ckpt_dir):
                    os.mkdir(best_ckpt_dir)
                ms.save_checkpoint(network, best_ckpt_path)


if __name__ == "__main__":
    main()

Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.
开始训练...
epoch :1,loss:1.1161618
epoch :2,loss:0.7576737
epoch :3,loss:0.61485964
epoch :4,loss:0.8425347
epoch :5,loss:2.041541
acc:0.9369469026548672
epoch :6,loss:0.42178908
epoch :7,loss:1.5010157
epoch :8,loss:0.44350916
epoch :9,loss:0.76472414
epoch :10,loss:0.26330495
acc:0.956858407079646
epoch :11,loss:0.114679135
epoch :12,loss:0.43888614
epoch :13,loss:0.11739351
epoch :14,loss:0.66906965
epoch :15,loss:0.37895548
acc:0.9845132743362832
epoch :16,loss:0.27083865
epoch :17,loss:0.43173543
epoch :18,loss:0.54853845
epoch :19,loss:0.0999575
epoch :20,loss:0.26559213
acc:0.9889380530973452
epoch :21,loss:0.6725527
epoch :22,loss:0.0793021
epoch :23,loss:0.47825706
epoch :24,loss:0.011841968
epoch :25,loss:0.43664354
acc:0.9900442477876106
epoch :26,loss:0.073056474
epoch :27,loss:0.008096693
epoch :28,loss:0.4137097
epoch :29,loss:0.21654691
epoch :30,loss:0.3582871
acc:0.99446902654867